# Finding the best locations for new pharmacies (IBM Data Science Capstone Project)

## Data

To perform our analysis, we will need some data. Luckily, we live in the information age and there is a wealth of data available online. For this project, we will be using two data sources, Foursquare and Open Data Austria.

### Foursquare

Foursquare specializes in location based data that is enriched with user based imagery and reviews. For our project, we will pull data from Foursquare that allows us to find the current location of pharmacies and match them up with the various districts of Vienna. Using this data, we'll be able to calculate densities, like persons per pharmacy, today and 15 years in the future, which allows us to recommend locations where additional pharmacies might be needed.

Note to non-German speaking readers: "Apotheke"/"Wien" are the German words for "Pharmacy"/"Vienna", should you encounter them. 

In [1]:
%%html
<iframe src="https://de.foursquare.com/explore?mode=url&near=Wien%2C%20%C3%96sterreich&nearGeoId=72057594040689305&q=apotheke" width="1280" height="600"></iframe>

### Open Data Austria

[Open Data Austria](https://www.data.gv.at) is a website operated by the administration of Vienna, to make their data publicly available as Open Data. We will be mostly pulling census data from there, which includes current number of inhabitants per district as well as predictions 15 years into the future.

- Population projection by sex and age groups for districts in Vienna 2014 to 2034: https://www.wien.gv.at/statistik/ogd/vie_301.csv
- Population projection by sex and country of birth for districts in Vienna 2014 to 2034: https://www.wien.gv.at/statistik/ogd/vie_302.csv
- Pharmacy locations Vienna: https://data.wien.gv.at/daten/geo?service=WFS&request=GetFeature&version=1.1.0&typeName=ogdwien:APOTHEKEOGD&srsName=EPSG:4326&outputFormat=csv
- Vienna District boundaries: https://data.wien.gv.at/daten/geo?service=WFS&request=GetFeature&version=1.1.0&typeName=ogdwien:BEZIRKSGRENZEOGD&srsName=EPSG:4326&outputFormat=json

In [2]:
import pandas as pd
df1 = pd.read_csv('https://www.wien.gv.at/statistik/ogd/vie_301.csv', sep=';', header=1, encoding='latin-1')
print(df1.shape)
df1.sample(5)

(966, 18)


,NUTS1,NUTS2,NUTS3,DISTRICT_CODE,SUB_DISTRICT_CODE,SEX,AGE_00_02,AGE_03_05,AGE_06_09,AGE_10_14,AGE_15_19,AGE_20_24,AGE_25_29,AGE_30_44,AGE_45_59,AGE_60_74,AGE_75+,REF_DATE
143,AT1,AT13,AT130,90600,.,1,553,522,532,605,823,1074,1378,4225,3453,2115,863,20200101
173,AT1,AT13,AT130,91300,.,1,607,586,887,1161,1351,1672,1742,4609,5280,3578,2470,20210101
12,AT1,AT13,AT130,91300,.,1,746,716,905,1141,1180,1390,1463,4664,5183,4266,1987,20140101
362,AT1,AT13,AT130,91800,.,1,730,713,950,1230,1401,1741,1872,5234,4876,3598,1724,20290101
333,AT1,AT13,AT130,91200,.,1,1594,1544,2033,2509,2898,3490,3672,10690,9439,7110,3265,20280101


In [3]:
df2 = pd.read_csv('https://www.wien.gv.at/statistik/ogd/vie_302.csv', sep=';', header=1, encoding='latin-1')
print(df2.shape)
df2.sample(5)

(483, 11)


,NUTS1,NUTS2,NUTS3,DISTRICT_CODE,SUB_DISTRICT_CODE,POP_TOTAL,POP_MEN,POP_WOMEN,POP_NATIVE,POP_FOREIGN,REF_Date
412,AT1,AT13,AT130,92200,.,217402,103229,114172,160594,56808,20310513
15,AT1,AT13,AT130,91600,.,99742,48807,50935,60772,38970,20140116
9,AT1,AT13,AT130,91000,.,185325,90499,94826,115739,69586,20140110
325,AT1,AT13,AT130,90400,.,33841,16256,17584,20036,13805,20280426
349,AT1,AT13,AT130,90500,.,58586,28512,30075,32580,26007,20290450


In [4]:
df3 = pd.read_csv('https://data.wien.gv.at/daten/geo?service=WFS&request=GetFeature&version=1.1.0&typeName=ogdwien:APOTHEKEOGD&srsName=EPSG:4326&outputFormat=csv', sep=',', header=0, encoding='utf8')
print(df3.shape)
df3.sample(5)

(329, 13)


,FID,OBJECTID,SHAPE,BEZEICHNUNG,BEZIRK,STRASSE,ONR,ADRESSE,TELEFON,FAX,EMAIL,WEBLINK1,SE_ANNO_CAD_DATA
32,APOTHEKEOGD.fid-2dd4c72f_16cc8a9d95c_7bf8,792376,POINT (16.366473047340573 48.1667319094049),Heilfried-Apotheke,10,Van der Nüll-Gasse,110,"10., Van der Nüll-Gasse 110",01 604 44 80,01 604 44 80,apotheke@heilfriedapotheke.at,NaN,NaN
199,APOTHEKEOGD.fid-2dd4c72f_16cc8a9d95c_7c9f,792175,POINT (16.37009118404771 48.23562363402708),Arkaden-Apotheke,20,Jägerstraße,69,"20., Jägerstraße 69",01 350 72 72,01 350 72 72,info@arkadenapotheke.at,NaN,NaN
229,APOTHEKEOGD.fid-2dd4c72f_16cc8a9d95c_7cbd,792206,POINT (16.26676992390248 48.18715095959116),San Vito Apotheke,13,Einsiedeleigasse,10,"13., Einsiedeleigasse 10",01 877 86 78,01 877 86 78 77,info@sanvito.at,NaN,NaN
128,APOTHEKEOGD.fid-2dd4c72f_16cc8a9d95c_7c58,792302,POINT (16.33786704249701 48.20846287335645),Thalia-Apotheke,16,Lerchenfelder Gürtel,35,"16., Lerchenfelder Gürtel 35",01 492 08 92,01 492 08 92,thalia.apo@aon.at,NaN,NaN
268,APOTHEKEOGD.fid-2dd4c72f_16cc8a9d95c_7ce4,792244,POINT (16.284916203179943 48.14646151526933),Birken-Apotheke,23,Khemetergasse,8,"23., Khemetergasse 8",01 888 51 44,01 888 51 44,birkenapotheke@aon.at,NaN,NaN
